# **ETL del archivo Homicidios_Victimas**

## **1.Cargando Las Librerías Necesarias**

In [ ]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import datetime #Cargaremos datetime para manejar fechas",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive"
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url01="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/homicidios.xlsx"
url02="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/homicidios_victimas.csv"

## **2.Fase ETL**

### 2.1.Carga de Datos

In [ ]:
df_homicidios_victimas=pd.read_excel(url01,sheet_name="VICTIMAS")

In [ ]:
df_homicidios_victimas.to_csv(url02,index=True,header=True,encoding='utf-8')

In [ ]:
df_homicidios_victimas=pd.read_csv(url02,encoding='utf-8')

In [ ]:
df_homicidios_victimas.head()

,Unnamed: 0,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


### 2.2.Analisis Descriptivo De Los Datos

#### 2.2.1.Funciones para el analisis descriptivo de los datos

In [ ]:
def check_df(dataset, head = 5):

  """
  Funcion creada para tener una vista general de la base de datos
  """

  print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

  print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.info(), end = '\n'*2)# Informacion general

  print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
  print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

#### 2.2.2.Descripcion De Los Datos

In [ ]:
check_df(df_homicidios_victimas)

******************************Forma de la base de datos******************************

(717, 11)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           717 non-null    int64 
 1   ID_hecho             717 non-null    object
 2   FECHA                717 non-null    object
 3   AAAA                 717 non-null    int64 
 4   MM                   717 non-null    int64 
 5   DD                   717 non-null    int64 
 6   ROL                  717 non-null    object
 7   VICTIMA              717 non-null    object
 8   SEXO                 717 non-null    object
 9   EDAD                 717 non-null    object
 10  FECHA_FALLECIMIENTO  717 non-null    object
dtypes: int64(4), object(7)
memory usage: 61.7+ KB
None

**

### 2.3.Transformacion De Los Datos

#### 2.3.1.Funciones para la transformacion de los datos

In [ ]:
def reemplazar_nan(value):
  '''

  La funcion replace_value busca reemplaza las palabras como 'SD' , 'sd o
  No Especificada' por
  valores NaN

  '''
  if value in ('SD', 'sd', 'No Especificada'):
      return float(np.nan)
  else:
      return value

#### 2.3.2.Transformacion de los datos

In [ ]:
df_homicidios_victimas.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
df_homicidios_victimas['FECHA_FALLECIMIENTO'] = df_homicidios_victimas['FECHA_FALLECIMIENTO'].apply(
    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date() if x.count("-") >= 2 else (datetime.datetime.strptime(x, '%d/%m/%Y').date() if x.count("/")>=2 else float(np.nan)))

In [ ]:
df_homicidios_victimas=df_homicidios_victimas.applymap(reemplazar_nan)

In [ ]:
df_homicidios_victimas['FECHA'].value_counts()

2017-02-26    3
2016-02-28    3
2017-11-19    3
2017-12-11    3
2018-04-27    3
             ..
2017-11-23    1
2017-11-17    1
2017-11-16    1
2017-11-07    1
2021-12-15    1
Name: FECHA, Length: 598, dtype: int64

In [ ]:
df_homicidios_victimas[df_homicidios_victimas['ROL'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,NaN,NaN,NaN,NaN,NaN
39,2016-0052,2016-04-20,2016,4,20,NaN,MOTO,NaN,NaN,NaN
63,2016-0085,2016-06-29,2016,6,29,NaN,MOTO,MASCULINO,NaN,NaN
77,2016-0101,2016-08-07,2016,8,7,NaN,NaN,MASCULINO,67,NaN
89,2016-0115,2016-09-02,2016,9,2,NaN,NaN,MASCULINO,NaN,NaN
141,2016-0174,2016-12-27,2016,12,27,NaN,NaN,NaN,NaN,NaN
167,2017-0029,2017-03-07,2017,3,7,NaN,NaN,MASCULINO,34,2017-03-07
208,2017-0074,2017-06-04,2017,6,4,NaN,NaN,MASCULINO,70,2017-06-04
221,2017-0089,2017-07-13,2017,7,13,NaN,NaN,MASCULINO,23,NaN
280,2017-0155,2017-12-12,2017,12,12,NaN,NaN,MASCULINO,77,NaN


In [ ]:
df_homicidios_victimas['ROL'].value_counts()

CONDUCTOR               330
PEATON                  267
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
Name: ROL, dtype: int64

In [ ]:
df_homicidios_victimas[df_homicidios_victimas['VICTIMA'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,NaN,NaN,NaN,NaN,NaN
77,2016-0101,2016-08-07,2016,8,7,NaN,NaN,MASCULINO,67,NaN
89,2016-0115,2016-09-02,2016,9,2,NaN,NaN,MASCULINO,NaN,NaN
93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,NaN,FEMENINO,NaN,NaN
141,2016-0174,2016-12-27,2016,12,27,NaN,NaN,NaN,NaN,NaN
167,2017-0029,2017-03-07,2017,3,7,NaN,NaN,MASCULINO,34,2017-03-07
208,2017-0074,2017-06-04,2017,6,4,NaN,NaN,MASCULINO,70,2017-06-04
221,2017-0089,2017-07-13,2017,7,13,NaN,NaN,MASCULINO,23,NaN
280,2017-0155,2017-12-12,2017,12,12,NaN,NaN,MASCULINO,77,NaN


In [ ]:
df_homicidios_victimas[df_homicidios_victimas['SEXO'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
36,2016-0049,2016-04-17,2016,4,17,NaN,NaN,NaN,NaN,NaN
39,2016-0052,2016-04-20,2016,4,20,NaN,MOTO,NaN,NaN,NaN
108,2016-0136,2016-10-25,2016,10,25,CONDUCTOR,MOTO,NaN,NaN,NaN
121,2016-0151,2016-11-18,2016,11,18,PEATON,PEATON,NaN,NaN,NaN
138,2016-0171,2016-12-25,2016,12,25,CONDUCTOR,MOTO,NaN,NaN,NaN
141,2016-0174,2016-12-27,2016,12,27,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_homicidios_victimas[df_homicidios_victimas['EDAD'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,NaN,2016-03-30
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,NaN,NaN
35,2016-0048,2016-04-15,2016,4,15,PEATON,PEATON,FEMENINO,NaN,NaN
36,2016-0049,2016-04-17,2016,4,17,NaN,NaN,NaN,NaN,NaN
39,2016-0052,2016-04-20,2016,4,20,NaN,MOTO,NaN,NaN,NaN
55,2016-0077,2016-06-13,2016,6,13,PEATON,PEATON,FEMENINO,NaN,NaN
63,2016-0085,2016-06-29,2016,6,29,NaN,MOTO,MASCULINO,NaN,NaN
72,2016-0096,2016-07-25,2016,7,25,CONDUCTOR,MOTO,MASCULINO,NaN,NaN
89,2016-0115,2016-09-02,2016,9,2,NaN,NaN,MASCULINO,NaN,NaN
93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,NaN,FEMENINO,NaN,NaN


In [ ]:
df_homicidios_victimas[df_homicidios_victimas['FECHA_FALLECIMIENTO'].isnull()]

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,NaN
16,2016-0022,2016-02-21,2016,2,21,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,41,NaN
19,2016-0027,2016-02-28,2016,2,28,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,34,NaN
22,2016-0031,2016-03-08,2016,3,8,CONDUCTOR,MOTO,MASCULINO,21,NaN
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
221,2017-0089,2017-07-13,2017,7,13,NaN,NaN,MASCULINO,23,NaN
242,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,1,NaN
246,2017-0115,2017-09-19,2017,9,19,CONDUCTOR,MOTO,MASCULINO,34,NaN
253,2017-0126,2017-10-14,2017,10,14,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,39,NaN


In [ ]:
df_homicidios_victimas

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18
714,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02


### 3.Exportacion

In [ ]:
path="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Data_Clean/homicidios_victimas.parquet.gzip"

In [ ]:
df_homicidios_victimas.to_parquet(path,compression="gzip")